In [1]:
import requests
import pandas as pd
import time

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
def get_posts(subreddit, number):
    url = 'https://api.pushshift.io/reddit/search/submission'
    params = {
            'subreddit': subreddit,
            'size': 250
        } # 'before' is added later in the while loop
    df = pd.DataFrame() # empty dataframe for concatenating
    returned = pd.DataFrame()
    while True: # Need a break out of this
        time.sleep(3)
        
        res = requests.get(url, params)
        
        data = res.json()
        posts = data['data']
        df = pd.DataFrame(posts)
        
        params['before'] = df.iloc[-1:,:]['created_utc']
        
        returned = pd.concat([returned, df[['subreddit','selftext','title']]], axis=0)
        returned.drop_duplicates(inplace=True)
        returned.dropna(inplace=True)
        #returned.drop(returned[returned['selftext'] =='[removed]'].index, inplace = True)
        
        
        if len(returned) > number:
            break
      
    returned.reset_index(inplace=True,drop=True)
    return returned[:number]

In [4]:
intermittentfasting = get_posts('intermittentfasting',15000)

In [5]:
intermittentfasting.isnull().sum()

subreddit    0
selftext     0
title        0
dtype: int64

In [6]:
intermittentfasting[(intermittentfasting['selftext']=='[removed]')].count()

subreddit    2710
selftext     2710
title        2710
dtype: int64

In [7]:
intermittentfasting.drop(intermittentfasting[(intermittentfasting['selftext'] =='[removed]')\
                                             |(intermittentfasting['selftext'] =='')].index, inplace = True)

In [8]:
intermittentfasting = pd.melt(intermittentfasting,id_vars='subreddit', value_vars=['selftext', 'title'],
                              var_name='selftext_or_title', value_name='text')

In [9]:
intermittentfasting.drop(intermittentfasting[(intermittentfasting['text'].str.split().str.len())<=20].index, inplace = True)

In [10]:
intermittentfasting.drop('selftext_or_title', axis = 1, inplace = True)

In [11]:
intermittentfasting.shape

(7010, 2)

In [12]:
intermittentfasting[(intermittentfasting['text']=='')].count()

subreddit    0
text         0
dtype: int64

In [13]:
intermittentfasting.shape

(7010, 2)

In [14]:
intermittentfasting.head()

,subreddit,text
0,intermittentfasting,Hello everyone with a uterus! I started interm...
1,intermittentfasting,I have a question. I'm on 16:8 and it's going ...
2,intermittentfasting,was it successful for you? did you incorporate...
4,intermittentfasting,I started doing 16:8 for a few months and lost...
5,intermittentfasting,I hope everyone has a great Thanksgiving tomor...


In [15]:
keto = get_posts('keto',15000)

In [16]:
keto.isnull().sum()

subreddit    0
selftext     0
title        0
dtype: int64

In [17]:
keto[(keto['selftext']=='[removed]')].count()

subreddit    6041
selftext     6041
title        6041
dtype: int64

In [18]:
keto.drop(keto[(keto['selftext'] =='[removed]')|(keto['selftext'] =='')].index, inplace = True)

In [19]:
keto = pd.melt(keto,id_vars='subreddit', value_vars=['selftext', 'title'],
               var_name='selftext_or_title', value_name='text')

In [20]:
keto.drop(keto[(keto['text'].str.split().str.len())<=20].index, inplace = True)

In [21]:
keto.drop('selftext_or_title', axis = 1, inplace = True)

In [22]:
keto[(keto['text']=='')].count()

subreddit    0
text         0
dtype: int64

In [23]:
keto.shape

(9011, 2)

In [24]:
keto.head()

,subreddit,text
0,keto,So I started keto 3 weeks ago and I never had ...
1,keto,I made this Keto breakfast egg muffin. No suga...
2,keto,So I’m down 25LBs in 5 weeks on a semi-strict ...
3,keto,"Hi guys!\nI’m 28y, fit, and do martial arts ab..."
4,keto,Is there any success with reds in particular? ...


In [25]:
intermittentfasting.to_csv('./data/intermittentfasting.csv', index=False)

In [26]:
keto.to_csv('./data/keto.csv', index=False)

In [27]:
combined = pd.concat([intermittentfasting,keto])
combined.shape

(16021, 2)

In [28]:
combined.to_csv('./data/combined.csv', index=False)